In [1]:
import matplotlib.pyplot as plt

from DP_GP import plot
from DP_GP import utils
from DP_GP import core
from DP_GP import cluster_tools

import pandas as pd
import numpy as np
import numpy.linalg as nl
import scipy
import GPy

# import standard library dependencies:
import collections
import time
import copy
import argparse
import os

## Input arguments

In [8]:
# list of count matrice(s) filepath
gene_expression_matrix = ['../../data/unstim_WT_counts_df.txt']
# filepath to save the results
output = "./test"

## Optional sampling Arguments
# Maximum number of Gibbs sampling iterations. default=1000
max_num_iters = 1000
# Take every sth sample during Gibbs iterations to ensure independence between samples. default=3
thinning_param = 3
# Specify the optimization technique used to update GP hyperparameters
# lbfgsb = L-BFGS-B 
# fmin_tnc = truncated Newton algorithm 
# simplex = Nelder-Mead simplex 
# scg = stochastic conjugate gradient 
# default = 'lbfgsb'
optimizer = 'lbfgsb'
# default=1000
max_iters = 1000



In [15]:
##############################################################################
#
#  Optional hyperprior arguments
#
##############################################################################


# sigma_n2_shape is shape parameter for the inverse gamma prior on the cluster noise variance. default=12
sigma_n2_shape = 12
# sigma_n2_rate is rate parameter for the inverse gamma prior on the cluster noise variance. default=2
sigma_n2_rate = 2
# for length scale [default=0]
length_scale_mu = 0
# for length scale [default=1]
length_scale_sigma = 1
# for signal variance [default=0]
sigma_f_mu = 0
# for signal variance [default=1]
sigma_f_sigma = 1

## Reading the count matrix

In [11]:
## Optional input transformation arguments

# Set this flag if the header contains true time values (e.g. 0, 0.5, 4, 8,...)
# and it is desired that the covariance kernel recognizes the true
# time spacing between sampling points, which need not be constant.
# Otherwise, it is assumed that the sampling times are equally spaced, 
# or in other words, that the rate of change in expression is roughly equivalent
# between all neighboring time points.
# default=False
true_times = False 

# Set this flag if you desire the gene expression data to be clustered
# without scaling (do not divide by standard deviation).
# default=False
unscaled = False

#Set this flag if you desire the gene expression data to be clustered
#without mean-centering (do not subtract mean).
#default=False
do_not_mean_center = False

In [12]:
gene_expression_matrix, gene_names, t, t_labels = \
core.read_gene_expression_matrices(gene_expression_matrix, 
                                   true_times, 
                                   unscaled, 
                                   do_not_mean_center)

In [17]:
sigma_n2_shape, sigma_n2_rate = sigma_n2_shape, sigma_n2_rate
sigma_n = np.sqrt(1 / ((sigma_n2_shape + 1) * sigma_n2_rate))